In [ ]:
from Enceladus.jobs import TestingPipeline

TestingPipeline(
    data_path='/home/cam/Documents/database_tools/data-2022-11-08/mimic3/',
    scaler_path='/home/cam/Documents/database_tools/data-2022-11-08/mimic3/scalers_MinMax.pkl',
    scaler_type='MinMax',
    model_path='heartfelt/Enceladus/model-vital-bee-206:v19',
).run()

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl

with open('model-vital-bee-206:v19_error.pkl', 'rb') as f:
    data = pkl.load(f)

print(np.mean(data['sbp_abs_err']), np.mean(data['dbp_abs_err']))

In [ ]:
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

fig = go.Figure()
fig.add_trace(go.Histogram(x=data['sbp_err'], name='SBP Error'))
fig.add_trace(go.Histogram(x=data['dbp_err'], name='DBP Error'))

fig.update_layout(
        barmode='overlay',
        title={
            'text': 'Model Prediction Error Histogram',
            'font': {'size': 20},
        },
        xaxis={'title': 'Error (mmHg)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Number of Samples', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig
fig.update_traces(opacity=0.75)

fig

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from plotly.subplots import make_subplots

with open('model-vital-bee-206:v19_predictions.pkl', 'rb') as f:
    data = pkl.load(f)
    
ppg, vpg, apg, abp, pred = data.values()

In [ ]:
i = 18

In [ ]:
fig = make_subplots(rows=2, cols=1, start_cell='top-left')

# Add inputs
fig.add_scatter(
    y=ppg[i, :],
    name='PPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=vpg[i, :] * 10,
    name='VPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=apg[i, :] * 50,
    name='APG',
    row=1,
    col=1,
)

# Add ABP and prediction
fig.add_scatter(
    y=abp[i, :],
    name='ABP (Ground Truth)',
    row=2,
    col=1
)
fig.add_scatter(
    y=pred[i, :],
    name='Prediction',
    row=2,
    col=1,
)

fig.update_layout(
        title={
            'text': 'Model Inputs (PPG, VPG, APG) and Module Output w/ True ABP',
            'font': {'size': 20},
        },
        xaxis2={'title': 'Time (samples)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Amplitude (V)', 'titlefont':{'size': 15}},
        yaxis2={'title': 'Amplitude (mmHg)', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig